# Neural Speech Generation for Shakespeare Play

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Set Up Environment

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## Load Data

In [3]:
fname = "../data/The-Comedy-of-Errors.txt"

with open(fname) as f: 
    document = f.read()

#print(document)

## Set Up Azure OpenAI

In [4]:
import os
import openai

# Set up Azure OpenAI
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

## Deploy a Model

In [5]:
# id of desired_model
desired_model = 'gpt-4-32k' 
desired_capability = 'chat_completion' # apply as completion, since gpt-4 is only released as chat in Azure OpenAI

# list models deployed with
deployment_id = None
result = openai.Deployment.list()

for deployment in result.data:
    if deployment["status"] != "succeeded":
        continue
    
    model = openai.Model.retrieve(deployment["model"])
    print(model)
    # check if desired_model is deployed, and if it has 'completion' capability
    if model["id"] == desired_model and model['capabilities'][desired_capability]:
        deployment_id = deployment["id"]
        
# if no model deployed, deploy one
if not deployment_id:
    print('No deployment with status: succeeded found.')

    # Deploy the model
    print(f'Creating a new deployment with model: {desired_model}')
    result = openai.Deployment.create(model=desired_model, scale_settings={"scale_type":"standard"})
    deployment_id = result["id"]
    print(f'Successfully created {desired_model} that supports text {desired_capability} with id: {deployment_id}.')
else:
    print(f'Found a succeeded deployment of "{desired_model}" that supports text {desired_capability} with id: {deployment_id}.')

{
  "capabilities": {
    "chat_completion": true,
    "completion": false,
    "embeddings": false,
    "fine_tune": false,
    "inference": true,
    "scale_types": []
  },
  "created_at": 1679356800,
  "deprecation": {
    "inference": 1742515200
  },
  "id": "gpt-4-32k",
  "lifecycle_status": "generally-available",
  "object": "model",
  "status": "succeeded",
  "updated_at": 1679356800
}
Found a succeeded deployment of "gpt-4-32k" that supports text chat_completion with id: gpt-4-32k.


## Identify Casts, Scenes, Acts, Dialogues, Synopsis with OpenAI

### Request API

In [6]:
def request_api(messages, deployment_id):
    response = openai.ChatCompletion.create(
        engine = deployment_id,
        messages = messages,
        temperature=0,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop='###')
    return response

### Prompt Design

In [7]:
content_system = "You are an assistant role is to extract entities and their associated values from a Shakespeare's play, then convert the extracted entities into json format."

example_user = "\nACT I\nSCENE I. On a ship at sea: a tempestuous noise\nof thunder and lightning heard.\nEnter a Master and a Boatswain\n\nMASTER\nBoatswain!\n\nBOATSWAIN\nHere, master: what cheer?\n\nMASTER\nGood, speak to the mariners: fall to't, yarely,\nor we run ourselves aground: bestir, bestir.\n\nExit\n\nEnter Mariners\n\nBOATSWAIN\nHeigh, my hearts! \n\nEnter ALONSO, SEBASTIAN, ANTONIO, FERDINAND, GONZALO, and others\n\nALONSO\nGood boatswain, have care. Where's the master?\n\nBOATSWAIN\nI pray now, keep below.\n\nANTONIO\nWhere is the master, boatswain?\n\nExit\n\n\nACT II\n\nSCENE I. Another part of the island.\nEnter ALONSO, SEBASTIAN, ANTONIO, GONZALO, ADRIAN, FRANCISCO, and others\n\nGONZALO\nBeseech you, sir, be merry; \n\nALONSO\nPrithee, peace.\n\nSEBASTIAN\nHe receives comfort like cold porridge.\n\nExit\n\nSCENE II. Another part of the island.\nEnter CALIBAN with a burden of wood. A noise of thunder heard\nCALIBAN\nAll the infections that the sun sucks up\nFrom bogs, fens, flats, on Prosper fall and make him\nBy inch-meal a disease! \n\nEnter TRINCULO\n\nLo, now, lo!\n\nExuent\n\n###\n\nExtract the entities and values from the Shakespeare's script play above. Then, convert the entities and values into json format. "

example_assistant = "{\n  \"Act I\": {\n    \"Scene I\": {\n      \"Location\": \"On a ship at sea\",\n      \"Setting\": \"A tempestuous noise of thunder and lightning heard\",\n      \"Characters\": [\n        \"Master\",\n        \"Boatswain\",\n        \"Mariners\",\n        \"Alonso\",\n        \"Sebastian\",\n        \"Antonio\",\n        \"Ferdinand\",\n        \"Gonzalo\"\n      ],\n      \"Enter\": [\n        \"Master\",\n        \"Boatswain\"\n      ],\n      \"Dialogues\": [\n        {\n          \"Speaker\": \"Master\",\n          \"Text\": \"Boatswain!\"\n        },\n        {\n          \"Speaker\": \"Boatswain\",\n          \"Text\": \"Here, master: what cheer?\"\n        },\n        {\n          \"Speaker\": \"Master\",\n          \"Text\": \"Good, speak to the mariners: fall to't, yarely, or we run ourselves aground: bestir, bestir.\"\n        },\n        {\n          \"Speaker\": \"Boatswain\",\n          \"Text\": \"Heigh, my hearts!\"\n        },\n        {\n          \"Speaker\": \"Alonso\",\n          \"Text\": \"Good boatswain, have care. Where's the master?\"\n        },\n        {\n          \"Speaker\": \"Boatswain\",\n          \"Text\": \"I pray now, keep below.\"\n        },\n        {\n          \"Speaker\": \"Antonio\",\n          \"Text\": \"Where is the master, boatswain?\"\n        }\n      ],\n      \"Exit\": [\n        \"Master\",\n        \"Boatswain\"\n      ]\n    }\n  },\n  \"Act II\": {\n    \"Scene I\": {\n      \"Location\": \"Another part of the island\",\n      \"Characters\": [\n        \"Alonso\",\n        \"Sebastian\",\n        \"Antonio\",\n        \"Gonzalo\",\n        \"Adrian\",\n        \"Francisco\"\n      ],\n      \"Enter\": [\n        \"ALONSO\",\n        \"SEBASTIAN\",\n        \"ANTONIO\",\n        \"GONZALO\",\n        \"ADRIAN\",\n        \"FRANCISCO\",\n        \"others\"\n      ],\n      \"Dialogues\": [\n        {\n          \"Speaker\": \"Gonzalo\",\n          \"Text\": \"Beseech you, sir, be merry;\"\n        },\n        {\n          \"Speaker\": \"Alonso\",\n          \"Text\": \"Prithee, peace.\"\n        },\n        {\n          \"Speaker\": \"Sebastian\",\n          \"Text\": \"He receives comfort like cold porridge.\"\n        }\n      ],\n      \"Exit\": [\n        \"ALONSO\",\n        \"SEBASTIAN\",\n        \"ANTONIO\",\n        \"GONZALO\",\n        \"ADRIAN\",\n        \"FRANCISCO\",\n        \"others\"\n      ]\n    },\n    \"Scene II\": {\n      \"Location\": \"Another part of the island\",\n      \"Setting\": \"A noise of thunder heard\",\n      \"Characters\": [\n        \"Caliban\",\n        \"Trinculo\"\n      ],\n      \"Enter\": [\n        \"Caliban with a burden of wood\",\n        \"Trinculo\"\n      ],\n      \"Dialogues\": [\n        {\n          \"Speaker\": \"Caliban\",\n          \"Text\": \"All the infections that the sun sucks up From bogs, fens, flats, on Prosper fall and make him By inch-meal a disease!\"\n        },\n        {\n          \"Speaker\": \"Caliban\",\n          \"Text\": \"Lo, now, lo!\"\n        }\n      ],\n      \"Exit\": [\n        \"Caliban\",\n        \"Trinculo\"\n      ]\n    }\n  }\n}"

### Get Structured Data

In [8]:
def get_structured_data(document, prompt_postfix, content_system, example_user, example_assistant, deployment_id):
    content = prompt_postfix.replace('<document>', document)
    messages = [{"role":"system","content":content_system},
                {"role":"user", "content":example_user},
                {"role":"assistant", "content":example_assistant},
                {"role":"user", "content":content}]; #print(messages)

    structured_data = request_api(messages, deployment_id)
    return structured_data

In [ ]:
# Prompt postfix
prompt_postfix = """ <document>
  \n###
  \nExtract the entities and values from the Shakespeare's script play above. Then, convert the entities and values into json format. 
"""
#print(prompt_postfix)

structured_data = get_structured_data(document, prompt_postfix, content_system, example_user, example_assistant, deployment_id)

In [ ]:
print(structured_data['choices'][0]['message']['content'])

## Generate Neural Speech

In [ ]:
import os
import azure.cognitiveservices.speech as speechsdk

speech_config = speechsdk.SpeechConfig(subscription=os.getenv('SPEECH_KEY'), region=os.getenv('SPEECH_REGION'))
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

# Text to synthesise 
text = "Hey, how are you?"

# Synthesise speech
result = speech_synthesizer.speak_text_async(text).get()

In [ ]:
def speech_synthesis_to_mp3_file():
    """performs speech synthesis to a mp3 file"""
    # Creates an instance of a speech config with specified subscription key and service region.
    speech_config = speechsdk.SpeechConfig(subscription=os.getenv('SPEECH_KEY'), region=os.getenv('SPEECH_REGION'))
    
    # Sets the synthesis output format.
    # The full list of supported format can be found here:
    # https://docs.microsoft.com/azure/cognitive-services/speech-service/rest-text-to-speech#audio-outputs
    speech_config.set_speech_synthesis_output_format(speechsdk.SpeechSynthesisOutputFormat.Audio16Khz32KBitRateMonoMp3)
    
    # Creates a speech synthesizer using file as audio output.
    # Replace with your own audio file name.
    file_name = "outputaudio.mp3"
    file_config = speechsdk.audio.AudioOutputConfig(filename=file_name)

    # Sets the synthesis voice name.
    voice = "Microsoft Server Speech Text to Speech Voice (en-US, JennyNeural)"  <--- update this
    speech_config.speech_synthesis_voice_name = voice

    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=file_config)

    try:
        #text = input()
        text = "Proceed, Solinus, to procure my fall, And by the doom of death end woes and all."
    except EOFError as e:
        print(e)
        exit()
        
    result = speech_synthesizer.speak_text_async(text).get()
    # Check result
    if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized for text [{}], and the audio was saved to [{}]".format(text, file_name))
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("Speech synthesis canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))

In [ ]:
speech_synthesis_to_mp3_file()

Speech synthesized for text [Proceed, Solinus, to procure my fall, And by the doom of death end woes and all.], and the audio was saved to [outputaudio.mp3]
